# Time-series CNN forecasting
[source](https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/)


#### 7 Options:

*Predict Binary Class*
- **Univariate CNN Models**
   1.  Raw data -> binary class
   2.  Only high pass -> binary class
- **Multivariate CNN Models**
   - **Multiple Input Series**
      3. Spike-sorted/single-unit -> binary class
   - **Multiple Parallel Series**

*Predict voltage instead of binary class, then feed to a seizure classification algorithm*
- **Multi-Step CNN Models**
   4. Raw data -> raw data 
   5. Only high pass -> only high pass
- **Multivariate Multi-Step CNN Models**
    - **Multiple Input Multi-Step Output**
       6. Spike-sorted/single-unit -> high pass
    - **Multiple Parallel Input and Multi-Step Output**
       7. Spike-sorted/single-unit -> Spike-sorted/single-unit
      
#### Also 
I could combine the two and feed the voltage prediction back into another CNN that spits out a binary class (instead of feeding it into a seizure classification algorithm).

This could be applied to options 4-7 to produce models 8-11 **for a total of 11 models to evaluate**.

And this is only the temporal CNN option. I could also try a [Multilayer Perceptron Model](https://machinelearningmastery.com/how-to-develop-multilayer-perceptron-models-for-time-series-forecasting/)
, [LSTM Model](https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/), Seq2Seq, TCN, GRU
and each of these could be trained the different ways as above as well. 3 x 11 = 33.

And there are more models and more combos. ahhh. hopefully a pattern in the options will quickly make clear what is best

I asked chatgpt what it thought and it was a helpful summary and agreement with my thoughts:

*What I asked it:* if given frequencies, is it easier for the model to learn the relationship between the current and future waves and predict the next parts of the waves which can then be classified as a siezure or not, or would it be easier to only tell it yes or no if the current waves came before a seizure so it couldn't learn a relationship between the current and future waves it could only learn a relationship betwen the current waves and a binary which seems like it has a less of a relationship

In [7]:
import numpy as np
from numpy import array

# Univariate CNN Model

In [14]:
x = np.arange(10, 100, 10)
x

array([10, 20, 30, 40, 50, 60, 70, 80, 90])

In [36]:
# source linked above perfectly explains why you're doing this

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [35]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence_univar(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


## Multiple Input Series

In [24]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [26]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

In [30]:
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


A 1D CNN model needs sufficient context to learn a mapping from an input sequence to an output value. **CNNs can support parallel input time series as separate channels, like red, green, and blue components of an image**. Therefore, *we need to split the data into samples maintaining the order of observations across the two input sequences*.

If we chose three input time steps,

`10, 15`

`20, 25`

`30, 35`

the first three time steps of each parallel series are provided as input to the model and the model associates this with the value in the output series at the third time step, in this case, `65`.

In [37]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [50]:
# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print("Number of samples:", X.shape[0],
      "\nNumber of timesteps:", X.shape[1]
      ,"\nNumber of parallel time series (or # of vars/features):", X.shape[2]
      ,"\nNumber of predictions:", y.shape[0], "\n")

# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

Number of samples: 7 
Number of timesteps: 3 
Number of parallel time series (or # of vars/features): 2 
Number of predictions: 7 

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


## Multiple Parallel Series

An alternate time series problem is the case where there are multiple parallel time series and a value must be predicted for each.

In [51]:
# define input sequence (same as previous example)
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [52]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

In [53]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

We may want to predict the value for each of the three time series for the next time step.

This might be referred to as multivariate forecasting.

Again, the data must be split into input/output samples in order to train a model.

Input:

`10, 15`

`20, 25`

`30, 35`

Output: `40, 45, 85`

The split_sequences() function below will split multiple parallel time series with rows for time steps and one series per column into the required input/output shape.

(Like univariate splitting but with multiple columns)

In [62]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [63]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


# Mix Two

## Multiple Input Multi-Step Output

In [64]:
# define input sequence (same as previous example)
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [65]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

In [66]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [67]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [68]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


## Multiple Parallel Input and Multi-Step Output

A problem with parallel time series may require the prediction of multiple time steps of each time series.

In [64]:
# define input sequence (same as previous example)
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [65]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

In [69]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [70]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [75]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print("\n\n",X[i], "\n\n",y[i])

(5, 3, 3) (5, 2, 3)


 [[10 15 25]
 [20 25 45]
 [30 35 65]] 

 [[ 40  45  85]
 [ 50  55 105]]


 [[20 25 45]
 [30 35 65]
 [40 45 85]] 

 [[ 50  55 105]
 [ 60  65 125]]


 [[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] 

 [[ 60  65 125]
 [ 70  75 145]]


 [[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] 

 [[ 70  75 145]
 [ 80  85 165]]


 [[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] 

 [[ 80  85 165]
 [ 90  95 185]]
